In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

## Problem Setting

In [2]:
# init
steepness = 50    # steepness factor
num_blocks = 1    # number of expression blocks
num_data = 9100   # number of data
test_size = 100   # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [3]:
# parameters as input data
p_low, p_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
p_train = np.random.uniform(p_low, p_high, (train_size, 1)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 1)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [4]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## Optimization Model

In [5]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks, timelimit=60)

In [23]:
rng = np.random.RandomState(17)
b = rng.normal(scale=1, size=(num_blocks))
q = rng.normal(scale=1, size=(num_blocks))
b, q

(array([0.27626589]), array([-1.85462808]))

In [28]:
params = {"p":p_test[0], "a":a_test[0]}
model.set_param_val(params)
solvals, objval = model.solve()

In [29]:
solvals

{'x': {0: -1.9926868120184897, 1: 4.0}}

In [30]:
objval

33.89757056905451

## Learnable Rounding

In [6]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [7]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 4             # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [8]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [9]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 633.96
Epoch 1, Validation Loss: 563.72
Epoch 2, Validation Loss: 52.03
Epoch 3, Validation Loss: 39.96
Epoch 4, Validation Loss: 37.59
Epoch 5, Validation Loss: 37.05
Epoch 6, Validation Loss: 37.39
Epoch 7, Validation Loss: 35.72
Epoch 8, Validation Loss: 38.21
Epoch 9, Validation Loss: 36.76
Epoch 10, Validation Loss: 38.36
Epoch 11, Validation Loss: 33.15
Epoch 12, Validation Loss: 32.57
Epoch 13, Validation Loss: 31.43
Epoch 14, Validation Loss: 31.64
Epoch 15, Validation Loss: 30.14
Epoch 16, Validation Loss: 28.77
Epoch 17, Validation Loss: 30.81
Epoch 18, Validation Loss: 28.81
Epoch 19, Validation Loss: 28.83
Epoch 20, Validation Loss: 29.86
Epoch 21, Validation Loss: 30.51
Epoch 22, Validation Loss: 30.46
Epoch 23, Validation Loss: 30.96
Epoch 24, Validation Loss: 30.43
Epoch 25, Validation Loss: 30.27
Epoch 26, Validation Loss: 29.60
Epoch 27, Validation Loss: 26.59
Epoch 28, Validation Loss: 26.70
Epoch 29, Validation Loss: 25.62
Epoch 30, Validati

In [10]:
p, a = p_test[0], a_test[0]
datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
              "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
              "name": "test"}

In [11]:
# infer
components.eval()
with torch.no_grad():
    for comp in components:
        datapoints.update(comp(datapoints))

In [12]:
datapoints

{'p': tensor([[6.0419]], device='cuda:0'),
 'a': tensor([[3.8258]], device='cuda:0'),
 'name': 'test',
 'x': tensor([[-1.1720,  2.9795]], device='cuda:0'),
 'x_rnd': tensor([[-1.8502,  3.0000]], device='cuda:0')}

In [13]:
p, x = rnd._extract_data(datapoints)
with torch.no_grad():
    h = rnd.layers(torch.cat(p+x, dim=-1))
h

tensor([[-0.6782,  9.4901]], device='cuda:0')

## Learnable Threshold

In [14]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [15]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 4             # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [16]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                          int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [17]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 245.18
Epoch 1, Validation Loss: 227.98
Epoch 2, Validation Loss: 47.26
Epoch 3, Validation Loss: 29.14
Epoch 4, Validation Loss: 24.36
Epoch 5, Validation Loss: 23.61
Epoch 6, Validation Loss: 23.70
Epoch 7, Validation Loss: 23.17
Epoch 8, Validation Loss: 23.34
Epoch 9, Validation Loss: 23.12
Epoch 10, Validation Loss: 23.36
Epoch 11, Validation Loss: 23.16
Epoch 12, Validation Loss: 23.04
Epoch 13, Validation Loss: 23.27
Epoch 14, Validation Loss: 22.90
Epoch 15, Validation Loss: 23.02
Epoch 16, Validation Loss: 22.70
Epoch 17, Validation Loss: 22.60
Epoch 18, Validation Loss: 22.73
Epoch 19, Validation Loss: 22.70
Epoch 20, Validation Loss: 22.94
Epoch 21, Validation Loss: 22.70
Epoch 22, Validation Loss: 22.98
Epoch 23, Validation Loss: 22.66
Epoch 24, Validation Loss: 22.56
Epoch 25, Validation Loss: 22.50
Epoch 26, Validation Loss: 22.52
Epoch 27, Validation Loss: 22.58
Epoch 28, Validation Loss: 22.41
Epoch 29, Validation Loss: 22.31
Epoch 30, Validati

In [18]:
p, a = p_test[0], a_test[0]
datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
              "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
              "name": "test"}

In [19]:
# infer
components.eval()
with torch.no_grad():
    for comp in components:
        datapoints.update(comp(datapoints))

In [20]:
datapoints

{'p': tensor([[6.0419]], device='cuda:0'),
 'a': tensor([[3.8258]], device='cuda:0'),
 'name': 'test',
 'x': tensor([[-1.1426,  3.0917]], device='cuda:0'),
 'x_rnd': tensor([[-1.8327,  3.0000]], device='cuda:0')}

In [21]:
p, x = rnd._extract_data(datapoints)
with torch.no_grad():
    h = rnd.layers(torch.cat(p+x, dim=-1))
h

tensor([[-0.6901, -1.8398]], device='cuda:0')

In [22]:
v = torch.sigmoid(h)
v

tensor([[0.3340, 0.1371]], device='cuda:0')